In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras import layers

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Carregar os dados

In [2]:
laptop_price = pd.read_pickle("../datasets/processed/laptop_price.pkl")
laptop_price = laptop_price.drop(columns=['product', 'cpu', 'gpu'])
laptop_price.head()

,company,typename,inches,screen_resolution,cpu_manufacturer,gpu_manufacturer,ram_gb,type_storage,hdd_gb,ssd_gb,hybrid_gb,flash_storage_gb,opsys,weight_kg,price_euros
0,Apple,Ultrabook,13.3,2560x1600,Intel,Intel,8,ssd,0,128,0,0,macOS,1.37,1339.69
1,Apple,Ultrabook,13.3,1440x900,Intel,Intel,8,flash storage,0,0,0,128,macOS,1.34,898.94
2,HP,Notebook,15.6,1920x1080,Intel,Intel,8,ssd,0,256,0,0,No OS,1.86,575.00
3,Apple,Ultrabook,15.4,2880x1800,Intel,AMD,16,ssd,0,512,0,0,macOS,1.83,2537.45
4,Apple,Ultrabook,13.3,2560x1600,Intel,Intel,8,ssd,0,256,0,0,macOS,1.37,1803.60


## Dividir dados em treino e teste

In [3]:
y = laptop_price['price_euros']
X = laptop_price.drop(columns='price_euros')

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2) 

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((833, 14), (833,), (209, 14), (209,), (261, 14), (261,))

## Normalizar dados

In [4]:
numeric_features = ['inches', 'ram_gb', 'hdd_gb', 'ssd_gb', 'hybrid_gb', 'flash_storage_gb', 'weight_kg']
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

categorical_features = ['company', 'typename', 'screen_resolution', 'cpu_manufacturer', 'gpu_manufacturer']
one_hot_enconder = OneHotEncoder(handle_unknown="ignore", sparse=False)

preprocessor = ColumnTransformer(
    transformers=[
        ("ohe", one_hot_enconder, categorical_features)
    ]
)

models = {'RandomForestRegressor': RandomForestRegressor(n_estimators=200), 
          'KneighborsRegressor':KNeighborsRegressor(), 
          'SVC': SVR(), 
          'DecisionTreeRegressor': DecisionTreeRegressor(),
          'LogisticRegression': LogisticRegression(max_iter=1000)}


for name, current_model in models.items():
    model = Pipeline(
        steps=[
            ("preprocessor", preprocessor), 
            ("scaler", StandardScaler()), 
            ("classifier", current_model)
        ]
    )
    
    model.fit(x_train, y_train)
    score = model.score(x_val, y_val)

    print(f"{name}: {score}")
    

RandomForestRegressor: 0.2733343296872962
KneighborsRegressor: 0.44584677769159364
SVC: -0.01554799288357156
DecisionTreeRegressor: 0.0044354754659087225


ValueError: Unknown label type: 'continuous'